In [1]:
import pandas as pd
import re 

Import breach data, format breach data

In [ ]:
months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
          'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
          'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

def format_dates(date):
  month = date[:3]
  num = months[month]
  return date[3:] + '-' + num + '-' + '01'

breaches = pd.read_csv('../data/breaches/breaches_information.csv', skiprows=range(1, 26))
breaches = breaches.rename(columns={'year   ': 'year'})
breaches['records lost'] = breaches['records lost'].str.replace(',', '')
breaches['records lost'] = pd.to_numeric(breaches['records lost'])
breaches = breaches.drop(columns=['Unnamed: 11'])
breaches['organisation_lower'] = breaches['organisation'].str.lower()
breaches = breaches.rename(columns={'date': 'breach_date'})
breaches['breach_date'] = breaches['breach_date'].str.replace(' ', '')
breaches['breach_date'] = pd.to_datetime(breaches['breach_date'].map(format_dates))
print(breaches.columns)

Index(['organisation', 'alternative name', 'records lost', 'year',
       'breach_date', 'story', 'sector', 'method', 'interesting story',
       'data sensitivity', 'displayed records', 'source name',
       '1st source link', '2nd source link', 'ID', 'organisation_lower'],
      dtype='object')


Import article data, format article data

In [3]:
articles_no_dupes = pd.read_csv('../data/article_data/joined_articles_companies_no_duplicates.csv')
articles_no_dupes = articles_no_dupes.rename(columns={'Date': 'article_date'})
articles_str_dates = articles_no_dupes['article_date'].astype(str)
articles_no_dupes['article_date'] = pd.to_datetime(articles_str_dates.map(lambda date: date[:4] + '-' + date[4:6] + '-' + date[6:]))
print(articles_no_dupes.columns)

articles_dupes = pd.read_csv('../data/article_data/joined_articles_companies.csv')
articles_dupes = articles_dupes.rename(columns={'Date': 'article_date'})
articles_str_dates = articles_dupes['article_date'].astype(str)
articles_dupes['article_date'] = pd.to_datetime(articles_str_dates.map(lambda date: date[:4] + '-' + date[4:6] + '-' + date[6:]))
print(articles_dupes.columns)

Index(['Unnamed: 0', 'name', 'domain', 'year founded', 'industry',
       'size range', 'locality', 'country', 'linkedin url',
       'current employee estimate', 'total employee estimate', 'article_date',
       'Publication', 'Headline', 'URL', 'BreachMentioned',
       'CompanyMentioned'],
      dtype='object')
Index(['Unnamed: 0', 'name', 'domain', 'year founded', 'industry',
       'size range', 'locality', 'country', 'linkedin url',
       'current employee estimate', 'total employee estimate', 'article_date',
       'Publication', 'Headline', 'URL', 'BreachMentioned',
       'CompanyMentioned'],
      dtype='object')


Calculate smallest interval that 2 breaches occurred for any single company

In [21]:
# Check shortest period (in days) between breaches experienced by one company (ends up being 90 days)
breaches_joined = pd.merge(breaches, breaches, on='organisation', how='inner')
breaches_joined_diff_dates = breaches_joined[breaches_joined['breach_date_x'] != breaches_joined['breach_date_y']]
min_breach_consecutive = min(abs(breaches_joined_diff_dates['breach_date_x'] - breaches_joined_diff_dates['breach_date_y']))
print(min_breach_consecutive)

# Restrict to article dates within max_days after the breach
max_days_since_breach = min_breach_consecutive - pd.Timedelta(1, 'd')
print(max_days_since_breach)

90 days 00:00:00
89 days 00:00:00


Join by exact company name, restricting `article_date` to within the interval after `breach_date`

In [5]:
# Join no duplicate articles
joined_no_dupes = pd.merge(breaches, articles_no_dupes, left_on='organisation_lower', right_on='name', how='inner')
joined_no_dupes = joined_no_dupes[joined_no_dupes['breach_date'] < joined_no_dupes['article_date']]
joined_no_dupes = joined_no_dupes[joined_no_dupes['article_date'] - joined_no_dupes['breach_date'] < max_days_since_breach]
joined_no_dupes.to_csv('../data/article_company_breach/joined_articles_company_breaches_no_duplicates.csv')

# Join duplicate articles exist
joined_dupes = pd.merge(breaches, articles_dupes, left_on='organisation_lower', right_on='name', how='inner')
joined_dupes = joined_dupes[joined_dupes['breach_date'] < joined_dupes['article_date']]
joined_dupes = joined_dupes[joined_dupes['article_date'] - joined_dupes['breach_date'] < max_days_since_breach]
joined_dupes.to_csv('../data/article_company_breach/joined_articles_company_breaches.csv')

# TODO: find which articles weren't matched

No article is matched to more than one data breach, so if an article references a past breach in addition to a newer breach, it will only match to the newer breach.

In [ ]:
companies_data = pd.read_csv("../data/companies.csv")
breaches = pd.read_csv('../data/breaches/breaches_information.csv', skiprows=range(1, 26))

months = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
          'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
          'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

def format_dates(date):
  month = date[:3]
  num = months[month]
  return date[3:] + '-' + num + '-' + '01'


breaches = breaches.rename(columns={'year   ': 'year'})
breaches['records lost'] = breaches['records lost'].str.replace(',', '').astype(int)
breaches['records lost'] = breaches['records lost'] = pd.to_numeric(breaches['records lost'])
breaches = breaches.drop(columns=['Unnamed: 11'])
breaches['organisation_lower'] = breaches['organisation'].str.lower()
breaches = breaches.rename(columns={'date': 'breach_date'})
breaches['breach_date'] = breaches['breach_date'].str.replace(' ', '')
breaches['breach_date'] = pd.to_datetime(breaches['breach_date'].map(format_dates))

def preprocess(text):
    # Remove punctuation and convert to lowercase
    if pd.isna(text):
        return ""
    return re.sub(r'[^\w\s]', '', text).lower()

# Preprocess data
companies_data['name_processed'] = companies_data['name'].apply(preprocess)
breaches['organisation_processed'] = breaches['organisation'].apply(preprocess)

# Perform the join on the processed columns
result_df = pd.merge(breaches, companies_data, left_on='organisation_processed', right_on='name_processed', how='left')

# Drop the helper columns 
result_df = result_df.drop(columns=['name_processed', 'organisation_processed'])

# Sort by 'name' (from companies_data)
result_df.sort_values(by=['organisation', 'name'])

result_df.to_csv("../data/article_company_breach/left_joined_breaches_companies.csv", index=False)

Do an left join on the data breaches table and companies tables, keeping all the data breach rows.

In [17]:
matched = pd.read_csv("../data/article_company_breach/left_joined_breaches_companies.csv")

# Fill NaN values in 'total employee estimate' with -1
matched['total employee estimate'] = matched['total employee estimate'].fillna(-1)

# drop rows with duplicate breaches
df_dedup = matched.loc[matched.groupby(['organisation', 'records lost', 'breach_date'])['total employee estimate'].idxmax()].reset_index(drop=True)

df_dedup.to_csv("../data/article_company_breach/left_joined_breaches_companies_no_duplicates.csv", index=False)

Remove duplicates based on 'total employee estimate'.

In [ ]:
articles_companies = pd.read_csv('../data/article_data/joined_articles_companies_no_duplicates.csv')
breaches_companies = pd.read_csv('../data/article_company_breach/left_joined_breaches_companies_no_duplicates.csv') 


articles_companies = articles_companies.rename(columns={'Date': 'article_date'})
articles_str_companies = articles_companies['article_date'].astype(str)
articles_companies['article_date'] = pd.to_datetime(articles_str_companies.map(lambda date: date[:4] + '-' + date[4:6] + '-' + date[6:]))

breaches_companies['breach_date'] = pd.to_datetime(breaches_companies['breach_date'])

# Join no duplicate articles
joined_no_dupes = pd.merge(breaches_companies, articles_companies, on=['Unnamed: 0','name','domain','year founded','industry','size range','locality','country','linkedin url','current employee estimate','total employee estimate'], how='left')
joined_no_dupes = joined_no_dupes[
    (joined_no_dupes['article_date'].isna()) | 
    (joined_no_dupes['breach_date'] < joined_no_dupes['article_date'])
]
joined_no_dupes = joined_no_dupes[
    (joined_no_dupes['article_date'].isna()) | 
    ((joined_no_dupes['article_date'] - joined_no_dupes['breach_date']) < max_days_since_breach)
]
joined_no_dupes.to_csv('../data/article_company_breach/left_joined_articles_company_breaches_no_duplicates.csv')

Left join articles with company and data breach table

In [26]:
df = companies_data = pd.read_csv("../data/article_company_breach/left_joined_articles_company_breaches_no_duplicates.csv")

df['Publication'] = df['Publication'].fillna('No Publication')

# Add a count column where 'publication' is NaN, the count will be 0
df['number of articles'] = df['Publication'].apply(lambda x: 0 if x == 'No Publication' else 1)

# Group by the specified columns and sum the 'number of articles' for each group
grouped = df.groupby(["organisation", "records lost", "year", "breach_date", "total employee estimate"])['number of articles'].sum().reset_index()

# Sort by 'total employee estimate' for better visualization
group_sizes_sorted = grouped.sort_values("total employee estimate")

group_sizes_sorted.to_csv('../data/article_company_breach/article_count_per_breach.csv')